In [ ]:
import tensorflow as tf

In [ ]:
class CustomConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding="VALID", **kwargs):
        super(CustomConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = (1, *strides, 1)
        self.padding = padding

    def build(self, input_shape):
        kernel_h, kernel_w = self.kernel_size
        input_dim = input_shape[-1]
        self.w = self.add_weight(name='kernel', 
                                 shape=(kernel_h, kernel_w, input_dim, self.filters),
                                 initializer='glorot_uniform', 
                                 trainable=True)  
        self.b = self.add_weight(name='bias', 
                                 shape=(self.filters,),
                                 initializer='zeros', 
                                 trainable=True)  

    def call(self, inputs):
        x = tf.nn.conv2d(inputs, self.w, self.strides, padding=self.padding) 
        x = tf.nn.bias_add(x, self.b)  
        x = tf.nn.relu(x)  
        return x

In [ ]:
def custom_categorical_crossentropy(y_true, y_pred):
    x = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return x

In [ ]:
class CustomCategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='custom_catrgorical_accuracy', **kwargs):
        super(CustomCategoricalAccuracy, self).__init__(name=name, **kwargs)
        self.correct = self.add_weight('correct_numbers', initializer='zeros')
        self.total = self.add_weight('total_numbers', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        values = tf.equal(y_true, y_pred)
        values = tf.cast(values, tf.float32)
        values_sum = tf.reduce_sum(values)
        num_values = tf.cast(tf.size(values), tf.float32)
        self.correct.assign_add(values_sum)  
        self.total.assign_add(num_values)  

    def result(self):
        return tf.math.divide_no_nan(self.correct, self.total)

    def reset_states(self):
        self.correct.assign(0.)
        self.total.assign(0.)

In [ ]:
class SaveModel(tf.keras.callbacks.Callback):
    def __init__(self, weights_file, monitor='loss', mode='min', save_weights_only=False):
        super(SaveModel, self).__init__()
        self.weights_file = weights_file
        self.monitor = monitor
        self.mode = mode
        self.save_weights_only = save_weights_only
        if mode == 'min':
            self.best = np.Inf
        else:
            self.best = -np.Inf
        
    def save_model(self):
        if self.save_weights_only:
            self.model.save_weights(self.weights_file)
        else:
            self.model.save(self.weights_file)

    def on_epoch_end(self, epoch, logs=None):
        monitor_value = logs.get(self.monitor)
        if self.mode == 'min' and monitor_value < self.best:
            self.save_model()
            self.best = monitor_value
        elif self.mode == 'max' and monitor_value > self.best:
            self.save_model()
            self.best = monitor_value

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from preprocessing import parse_aug_fn, parse_fn

In [ ]:
train_split, valid_split = ['train[:90%]', 'train[90%:]']
train_data, info = tfds.load("cifar10", split=train_split, with_info=True)
valid_data = tfds.load("cifar10", split=valid_split)
test_data = tfds.load("cifar10", split=tfds.Split.TEST)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteXVF02A/cifar10-train.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteXVF02A/cifar10-test.tfrecord


  0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE  
batch_size = 64  
train_num = int(info.splits['train'].num_examples / 10) * 9 

train_data = train_data.shuffle(train_num)  
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)


valid_data = valid_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

test_data = test_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [ ]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(inputs)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(256, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

model_1 = keras.Model(inputs, outputs, name='model-1')
model_1.summary()

Model: "model-1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 256)       295168    
                                                                 
 conv2d_3 (Conv2D)           (None, 9, 9, 128)         295040    
                                                           

In [ ]:
logs_dirs = 'lab6-logs'
model_cbk = keras.callbacks.TensorBoard(log_dir='lab6-logs')

model_dirs = logs_dirs + '/models'
os.makedirs(model_dirs, exist_ok=True)
save_model = tf.keras.callbacks.ModelCheckpoint(model_dirs + '/save.h5', 
                                                monitor='val_catrgorical_accuracy', 
                                                mode='max')

In [ ]:
model_1.compile(keras.optimizers.Adam(), 
                loss=keras.losses.CategoricalCrossentropy(from_logits=True), 
                metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model_1.fit(train_data,
            epochs=100, 
            validation_data=valid_data,
            callbacks=[model_cbk, save_model])

Epoch 1/100
704/704 [==============================] - 37s 28ms/step - loss: 2.1285 - categorical_accuracy: 0.1992 - val_loss: 1.8794 - val_categorical_accuracy: 0.3250
Epoch 2/100
704/704 [==============================] - 19s 28ms/step - loss: 1.9281 - categorical_accuracy: 0.2880 - val_loss: 1.8006 - val_categorical_accuracy: 0.3442
Epoch 3/100
704/704 [==============================] - 18s 26ms/step - loss: 1.8219 - categorical_accuracy: 0.3286 - val_loss: 1.5303 - val_categorical_accuracy: 0.4304
Epoch 4/100
704/704 [==============================] - 19s 26ms/step - loss: 1.7291 - categorical_accuracy: 0.3684 - val_loss: 1.4203 - val_categorical_accuracy: 0.4852
Epoch 5/100
704/704 [==============================] - 18s 26ms/step - loss: 1.6444 - categorical_accuracy: 0.4067 - val_loss: 1.3178 - val_categorical_accuracy: 0.5294
Epoch 6/100
704/704 [==============================] - 18s 26ms/step - loss: 1.5836 - categorical_accuracy: 0.4332 - val_loss: 1.2328 - val_categorical_acc

In [ ]:
inputs = keras.Input(shape=(32, 32, 3))
x = CustomConv2D(64, (3, 3))(inputs)
x = layers.MaxPool2D()(x)
x = CustomConv2D(128, (3, 3))(x)
x = CustomConv2D(256, (3, 3))(x)
x = CustomConv2D(128, (3, 3))(x)
x = CustomConv2D(64, (3, 3))(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

model_2 = keras.Model(inputs, outputs, name='model-2')
model_2.summary()

Model: "model-2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 custom_conv2d (CustomConv2D  (None, 30, 30, 64)       1792      
 )                                                               
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                             
                                                                 
 custom_conv2d_1 (CustomConv  (None, 13, 13, 128)      73856     
 2D)                                                             
                                                                 
 custom_conv2d_2 (CustomConv  (None, 11, 11, 256)      295168    
 2D)                                                       

In [ ]:
logs_dirs = 'lab6-logs'
model_cbk = keras.callbacks.TensorBoard(log_dir='lab6-logs')

model_dirs = logs_dirs + '/models'
os.makedirs(model_dirs, exist_ok=True)
custom_save_model = SaveModel(model_dirs + '/custom_save.h5', 
                              monitor='val_custom_catrgorical_accuracy', 
                              mode='max', 
                              save_weights_only=True)

In [ ]:
model_2.compile(keras.optimizers.Adam(), 
           loss=custom_categorical_crossentropy, 
           metrics=[CustomCategoricalAccuracy()])

model_2.fit(train_data,
            epochs=100, 
            validation_data=valid_data,
            callbacks=[model_cbk, custom_save_model])

Layer CustomConv2D has arguments ['self', 'filters', 'kernel_size', 'strides', 'padding']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Layer CustomConv2D has arguments ['self', 'filters', 'kernel_size', 'strides', 'padding']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Epoch 1/100
  5/704 [..............................] - ETA: 23s - loss: 2.3095 - custom_catrgorical_accuracy: 0.1250WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0186s vs `on_train_batch_end` time: 0.0269s). Check your callbacks.


702/704 [============================>.] - ETA: 0s - loss: 2.0429 - custom_catrgorical_accuracy: 0.2344

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1841: UserWarning: Metric CustomCategoricalAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


704/704 [==============================] - 27s 36ms/step - loss: 2.0427 - custom_catrgorical_accuracy: 0.2345 - val_loss: 1.7588 - val_custom_catrgorical_accuracy: 0.3700
Epoch 2/100
704/704 [==============================] - 25s 35ms/step - loss: 1.8397 - custom_catrgorical_accuracy: 0.3236 - val_loss: 1.4895 - val_custom_catrgorical_accuracy: 0.4562
Epoch 3/100
704/704 [==============================] - 18s 26ms/step - loss: 1.7240 - custom_catrgorical_accuracy: 0.3712 - val_loss: 1.4073 - val_custom_catrgorical_accuracy: 0.4862
Epoch 4/100
704/704 [==============================] - 19s 26ms/step - loss: 1.6400 - custom_catrgorical_accuracy: 0.4067 - val_loss: 1.3372 - val_custom_catrgorical_accuracy: 0.5262
Epoch 5/100
704/704 [==============================] - 19s 27ms/step - loss: 1.5697 - custom_catrgorical_accuracy: 0.4364 - val_loss: 1.2771 - val_custom_catrgorical_accuracy: 0.5456
Epoch 6/100
704/704 [==============================] - 18s 25ms/step - loss: 1.5014 - custom_catr

In [ ]:
model_1.load_weights(model_dirs+'/save.h5')
model_2.load_weights(model_dirs+'/custom_save.h5')

In [ ]:
loss_1, acc_1 = model_1.evaluate(test_data)
loss_2, acc_2 = model_2.evaluate(test_data)
loss = [loss_1, loss_2]  
acc = [acc_1, acc_2]
dict = {"Loss": loss, "Accuracy": acc}
pd.DataFrame(dict)

 13/157 [=>............................] - ETA: 1s - loss: 0.5514 - custom_catrgorical_accuracy: 0.8137

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1841: UserWarning: Metric CustomCategoricalAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


157/157 [==============================] - 2s 13ms/step - loss: 0.6312 - custom_catrgorical_accuracy: 0.8010


,Loss,Accuracy
0,0.719286,0.7672
1,0.631160,0.8010
